In [1]:
import os 
import pickle 
import numpy as np
import matplotlib.pyplot as plt
import math

import pandas as pd 
import seaborn as sb

# Load the pruned networks 

In [3]:
f = 'pruned_bias'

#Load the networks
sparseNetsFile = 'sparseNetworks.pkl'
sparseNetworks = pickle.load(open(os.path.join(f, sparseNetsFile), 'rb'))

#Load the losses
losses = pickle.load(open(os.path.join(f,'pruneLosses.pkl'),'rb'))

losses = np.array(losses)
losses = np.transpose(losses)

lossesDF = pd.DataFrame(losses, columns=['0%', '15%', '25%', '35%', '45%', '55%', '65%', '75%', '85%', '90%', '91%', 
                                        '92%', '93%', '94%', '95%', '96%', '97%', '98%' ])

# Find the masks

In [4]:
masks = []
for i in range(len(sparseNetworks)):
    net = sparseNetworks[i]
    sparsity = net[0]
    net = net[1]
    maskNet = []
    for i in np.arange(0,len(net),2):
        m = np.abs(net[i]) * np.reciprocal(np.abs(net[i]), where = np.abs(net[i])!=0)
        m = np.round(m)
        mask = m.astype(int)
        
        #Add bias term
        biasRow = np.abs(net[i+1]) * np.reciprocal(np.abs(net[i+1]), where = np.abs(net[i+1])!=0)
        mask = np.vstack([mask, biasRow])
        
        maskNet.append(mask)
        
    
    masks.append((sparsity, maskNet))

# Find the motif z-score

## Motif counting functions

### First-order motifs

In [23]:
def fom(m):
        '''
        Calculates the number of first-order motifs in the network (equivalent to the number of edges).
        
        Input(s): the mask of the pruned network, as a list of matrices
        Returns: FOM (total number of first-order motifs), FOMList (Number of weights and number of bias connections
                in each layer)
        '''
        FOM = 0
        FOMList = [[0,0],[0,0],[0,0],[0,0],[0,0]] #[[Num weights, num biases]] in each layer

        for i in range(len(m)): 
                #Count number of connections between weights
                w_connections = np.count_nonzero(m[i][0:-1])
                FOMList[i][0] = w_connections
                #Count number of connections from bias
                b_connections = np.count_nonzero(m[i][-1])
                FOMList[i][1] = b_connections

                connections = w_connections + b_connections
                FOM += connections
        return FOM, FOMList

### Second-order motifs

#### Diverging

In [24]:
def sodm(m):
    '''
    Calculates the number of second-order diverging motifs in the network.
    Also calculates the remaining number of nodes in the network. 
        
    Input(s): the mask of the pruned network, as a list of matrices
    Returns: SODM (total number of second-order diverging motifs in the network), numFC (Number of remaining nodes 
        with downstream output)
    '''
    
    SODM = 0
    numFC = [0,0,0,0,0,0] #Number of remaining nodes with downstream output

    for i in range(len(m)): 
        nodes = 0
        #Calculate second-order diverging motifs
        for row in m[i]:
            n = np.count_nonzero(row)
            if n >= 2:
                SODM += math.factorial(n)/(math.factorial(2)*math.factorial(n-2))
                    
            #also calculate number of remaining nodes
            if n > 0: 
                nodes += 1
                    
        numFC[i] = nodes
            
            #Count number of nodes in final layer 
        if i == 4: 
            nodes = 0 
            for row in m[i].T:
                n = np.count_nonzero(row)
                if n > 0 :
                    nodes += 1
            numFC[i+1] = nodes
    
    return SODM, numFC

#### Converging

In [25]:
def socm(m):
    '''
    Calculates the number of second-order converging motifs in the network.
        
    Input(s): the mask of the pruned network, as a list of matrices
    Returns: SOCM (total number of second-order converging motifs in the network)
    '''

    SOCM = 0

    for i in range(len(m)): 
        #Calculate second-order converging motifs
        for column in m[i].T:
            n = np.count_nonzero(column)
            if n >= 2:
                SOCM += math.factorial(n)/(math.factorial(2)*math.factorial(n-2))

    return SOCM

#### Chain

In [26]:
def sochain(m):
    '''
    Calculates the number of second-order chain motifs in the network.
        
    Input(s): the mask of the pruned network, as a list of matrices
    Returns: SOChain (total number of second-order chain motifs in the network)
    ''' 
    
    SOChain = 0 
    for i in range(len(m)): 
        #Calculate second-order chain motifs
        if i != 4: 
            #Exclude the bias by excluding the last row
            SOChain += np.count_nonzero(np.matmul(m[i][0:-1],m[i+1][0:-1]))
            
            #Add in the motifs from the bias terms 
            SOChain += np.count_nonzero(np.matmul(m[i][-1],m[i+1][0:-1]))
        else: 
            pass

    return SOChain

### Third-order motifs

#### Diverging

In [27]:
def todm(m):
    '''
    Calculates the number of third-order diverging motifs in the network.
        
    Input(s): the mask of the pruned network, as a list of matrices
    Returns: TODM (total number of third-order diverging motifs in the network)
    '''

    TODM = 0 
    for i in range(len(m)): 
        #Calculate third-order diverging motifs
        for row in m[i]:
            n = np.count_nonzero(row)
            if n >= 3:
                TODM += math.factorial(n)/(math.factorial(3)*math.factorial(n-3))

    return TODM

#### Converging

In [28]:
def tocm(m):
    '''
    Calculates the number of third-order converging motifs in the network.
        
    Input(s): the mask of the pruned network, as a list of matrices
    Returns: TOCM (total number of third-order converging motifs in the network)
    '''

    TOCM = 0 
    for i in range(len(m)): 
        #Calculate third-order converging motifs 
        for column in m[i].T:
            n = np.count_nonzero(column)
            if n >= 3:
                TOCM += math.factorial(n)/(math.factorial(2)*math.factorial(n-2))

    return TOCM

#### Chain

In [29]:
def tochain(m):
    '''
    Calculates the number of third-order chain motifs in the network.
        
    Input(s): the mask of the pruned network, as a list of matrices
    Returns: TOChain (total number of third-order chain motifs in the network)
    '''

    TOChain = 0
    for i in range(len(m)): 
        #Calculate third-order chain motifs 
        if i in (0,1,2): 
            #Count non-zero elements of (Layer 1 * Layer 2 * Layer 3)
            #Exclude the bias by excluding the last row
            m1 = np.matmul(m[i][0:-1],m[i+1][0:-1])
            TOChain += np.count_nonzero(np.matmul(m1,m[i+2][0:-1]))
                
            #Add in the motifs from the bias terms 
            mbias = np.matmul(m[i][-1],m[i+1][0:-1])
            TOChain += np.count_nonzero(np.matmul(mbias,m[i+2][0:-1]))
        else: 
            pass

    return TOChain

### Random networks

#### Build random network

In [12]:
def buildRandomNet(numFC, FOMList):
    '''
    Builds randomly connected network with the same number of weights and bias connections as the real network. 

    Input(s): numFC (Number of remaining nodes with downstream output), FOMList (Number of weights and number of 
        bias connections in each layer)
    Returns: the mask of the random network, as a list of matrices
    '''
    #random weight matrix = np.array(([1]*num connections between weights)+
    #                       [0]*(num possible connections - num connections between weights))
    #           numFC[0]-1 because we need to discount bias 
    r1 = np.array([1] * (FOMList[0][0]) + [0] * (((numFC[0]-1)*(numFC[1]-1))-(FOMList[0][0])))
    #random bias matrix = np.array(([1]*num connections between bias and next nodes)+
    #                       [0]*(num possible connections - num connections between bias and next nodes))

    #There is always a live bias, so number of possible connections between bias and next
    #   layer would be numFC[i+1]-1 (to remove bias).
    r1b = np.array([1] * FOMList[0][1] + [0] * ((numFC[1]-1)-FOMList[0][1]))

    r2 = np.array([1] * (FOMList[1][0]) + [0] * (((numFC[1]-1)*(numFC[2]-1))-(FOMList[1][0])))
    r3 = np.array([1] * (FOMList[2][0]) + [0] * (((numFC[2]-1)*(numFC[3]-1))-(FOMList[2][0])))
    r4 = np.array([1] * (FOMList[3][0]) + [0] * (((numFC[3]-1)*(numFC[4]-1))-(FOMList[3][0])))
    r5 = np.array([1] * (FOMList[4][0]) + [0] * (((numFC[4]-1)*(numFC[5]))-(FOMList[4][0]))) #no bias in last layer

    r2b = np.array([1] * FOMList[1][1] + [0] * ((numFC[2]-1)-FOMList[1][1]))
    r3b = np.array([1] * FOMList[2][1] + [0] * ((numFC[3]-1)-FOMList[2][1]))
    r4b = np.array([1] * FOMList[3][1] + [0] * ((numFC[4]-1)-FOMList[3][1]))
    r5b = np.array([1] * FOMList[4][1] + [0] * ((numFC[5])-FOMList[4][1]))
        
    np.random.shuffle(r1)
    np.random.shuffle(r2)
    np.random.shuffle(r3)
    np.random.shuffle(r4)
    np.random.shuffle(r5)

    np.random.shuffle(r1b)
    np.random.shuffle(r2b)
    np.random.shuffle(r3b)
    np.random.shuffle(r4b)
    np.random.shuffle(r5b)
            
    randomNet = [np.vstack([np.reshape(r1, (((numFC[0]-1),(numFC[1]-1)))),r1b]),
                np.vstack([np.reshape(r2, (((numFC[1]-1),(numFC[2]-1)))),r2b]),
                np.vstack([np.reshape(r3, (((numFC[2]-1),(numFC[3]-1)))),r3b]),
                np.vstack([np.reshape(r4, (((numFC[3]-1),(numFC[4]-1)))),r4b]),
                np.vstack([np.reshape(r5, (((numFC[4]-1),(numFC[5])))),r5b])]

    return randomNet

#### Find motifs for random networks 

In [13]:
def randomNetMotifs(randomNet):
    '''
    Finds all of the motifs for the random network.

    Input(s): the mask of the random network, as a list of matrices
    Returns: rFOM (random first-order motifs), rFOMList (remaining connections in each layer), rSODM (random second-oder 
        diverging motifs), rSOCM (random second-order converging motifs), rSOChain (random second-order chain motifs), 
        rTODM (random third-order diverging motifs), rTOCM (random third-order converging motifs), rTOChain (random 
        third-order chain motifs)
    '''

    rFOM, rFOMList = fom(randomNet)
    rSODM, rnumFC = sodm(randomNet)
    rSOCM = socm(randomNet)
    rSOChain = sochain(randomNet)
    rTODM = todm(randomNet)
    rTOCM = tocm(randomNet)
    rTOChain = tochain(randomNet)
    
    return rFOM, rFOMList, rSODM, rSOCM, rSOChain, rTODM, rTOCM, rTOChain

#### Average random motifs

In [14]:
def buildRandomMotifsDF(numFC, FOMList, numRand=1000):
    '''
    Builds dataframe of numRand number of random network motif counts. 
    To calculate the z-score, we need to compare the real network to many randomly generated networks. This function 
        adds all of that information to a dataframe. 

    Input(s): numFC (Number of remaining nodes with downstream output), FOMList (Number of weights and number of 
        bias connections in each layer), numRand=1000 (number of random networks we want to generate, default 1000)
    Returns: random network motif dataframe
    '''
    randomNetDF = pd.DataFrame(columns=['rSODM',
                                        'rSOCM',
                                        'rSOChain',
                                        'rTODM',
                                        'rTOCM', 
                                        'rTOChain'])

    for r in range(numRand):
        randomNet = buildRandomNet(numFC, FOMList)
        rFOM, rFOMList, rSODM, rSOCM, rSOChain, rTODM, rTOCM, rTOChain = randomNetMotifs(randomNet)

        rMotifs = [float(rSODM), float(rSOCM), float(rSOChain), float(rTODM), float(rTOCM), float(rTOChain)]
        randomNetDF.loc[len(randomNetDF.index)] = rMotifs

    return randomNetDF

## Z-score dataframe

In [15]:
masksTest = masks[0:5]

In [16]:
zscoreDF = pd.DataFrame(columns=['Sparsity Index', 'Masks',
                                '1-O motifs (real)', 
                                'S-O diverging motifs (real)', 'S-O converging motifs (real)', 
                                'S-O chain motifs (real)',  'T-O chain motifs (real)',
                                'T-O diverging motifs (real)', 'T-O converging motifs (real)',
                                
                                'Avg - S-O diverging motifs (random)', 'Avg - S-O converging motifs (random)', 
                                'Avg - S-O chain motifs (random)',  'Avg - T-O chain motifs (random)',
                                'Avg - T-O diverging motifs (random)', 'Avg - T-O converging motifs (random)',
                                 
                                'SD - S-O diverging motifs (random)', 'SD - S-O converging motifs (random)', 
                                'SD - S-O chain motifs (random)',  'SD - T-O chain motifs (random)',
                                'SD - T-O diverging motifs (random)', 'SD - T-O converging motifs (random)',
                                
                                'Z - S-O diverging motifs', 'Z - S-O converging motifs', 
                                'Z - S-O chain motifs',  'Z - T-O chain motifs',
                                'Z - T-O diverging motifs', 'Z - T-O converging motifs',
                                'Number of nodes in each layer with downstream output', 
                                'Number of connections in each layer'])

In [30]:
for (sparsity, m) in masksTest: 
    FOM, FOMList = fom(m)
    SODM, numFC = sodm(m)
    SOCM = socm(m)
    SOChain = sochain(m)
    TODM = todm(m)
    TOCM = tocm(m)
    TOChain = tochain(m)

    randomNetDF = buildRandomMotifsDF(numFC, FOMList, numRand=1000)

    AvgrSODM = randomNetDF['rSODM'].mean()
    AvgrSOCM = randomNetDF['rSOCM'].mean()
    AvgrSOChain = randomNetDF['rSOChain'].mean()
    AvgrTODM = randomNetDF['rTODM'].mean()
    AvgrTOCM = randomNetDF['rTOCM'].mean()
    AvgrTOChain = randomNetDF['rTOChain'].mean()

    SDrSODM = randomNetDF['rSODM'].std()
    SDrSOCM = randomNetDF['rSOCM'].std()
    SDrSOChain = randomNetDF['rSOChain'].std()
    SDrTODM = randomNetDF['rTODM'].std()
    SDrTOCM = randomNetDF['rTOCM'].std()
    SDrTOChain = randomNetDF['rTOChain'].std()

    ZSODM = (SODM - AvgrSODM)/SDrSODM
    ZSOCM = (SOCM - AvgrSOCM)/SDrSOCM
    ZSOChain = (SOChain - AvgrSOChain)/SDrSOChain
    ZTODM = (TODM - AvgrTODM)/SDrTODM
    ZTOCM = (TOCM - AvgrTOCM)/SDrTOCM
    ZTOChain = (TOChain - AvgrTOChain)/SDrTOChain

    zscoreData = [float(sparsity), m, 
                    float(FOM), 
                    float(SODM), float(SOCM), float(SOChain),
                    float(TODM), float(TOCM), float(TOChain),

                    float(AvgrSODM), float(AvgrSOCM), float(AvgrSOChain),
                    float(AvgrTODM), float(AvgrTOCM), float(AvgrTOChain),

                    float(SDrSODM), float(SDrSOCM), float(SDrSOChain),
                    float(SDrTODM), float(SDrTOCM), float(SDrTOChain),

                    float(ZSODM), float(ZSOCM), float(ZSOChain),
                    float(ZTODM), float(ZTOCM), float(ZTOChain),

                    numFC, FOMList]

    zscoreDF.loc[len(zscoreDF.index)] = zscoreData
    zscoreDF.to_csv('outputs/zscoreDF.csv')
    


In [31]:
zscoreDF.head()

,Sparsity Index,Masks,1-O motifs (real),S-O diverging motifs (real),S-O converging motifs (real),S-O chain motifs (real),T-O chain motifs (real),T-O diverging motifs (real),T-O converging motifs (real),Avg - S-O diverging motifs (random),...,SD - T-O diverging motifs (random),SD - T-O converging motifs (random),Z - S-O diverging motifs,Z - S-O converging motifs,Z - S-O chain motifs,Z - T-O chain motifs,Z - T-O diverging motifs,Z - T-O converging motifs,Number of nodes in each layer with downstream output,Number of connections in each layer
0,12.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -9.2233720368...",23142.0,325608.0,327710.0,143349.0,3193772.0,327645.0,11327.0,317715.879,...,592.601755,316.951757,16.144115,-168.938348,420.431530,17.381758,-168.936975,16.240197,"[9, 401, 401, 250, 6, 7]","[[281, 1], [11199, 4], [11199, 4], [447, 0], [..."
1,10.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",30576.0,563557.0,565781.0,162791.0,7222216.0,565682.0,12202.0,557970.482,...,723.072592,313.070009,8.607294,-113.554210,628.211826,12.041233,-113.519370,12.771955,"[9, 401, 400, 304, 9, 7]","[[469, 2], [15114, 6], [14400, 0], [575, 0], [..."
2,12.0,"[[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",23166.0,329361.0,330726.0,142518.0,3341234.0,330660.0,11756.0,318326.750,...,561.023536,306.529479,21.601967,-153.052876,344.963865,26.211785,-153.035815,18.638566,"[9, 401, 401, 262, 6, 7]","[[296, 0], [11199, 12], [11199, 6], [447, 0], ..."
3,11.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",26445.0,427054.0,425104.0,154905.0,4853515.0,425041.0,12101.0,414981.601,...,618.480796,322.637085,20.989577,-182.079987,539.898521,23.966671,-182.079993,17.360478,"[9, 401, 401, 267, 6, 7]","[[319, 1], [12799, 7], [12799, 1], [511, 0], [..."
4,12.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",23141.0,322916.0,328395.0,143664.0,3115825.0,328327.0,11521.0,317144.013,...,601.983493,320.700889,11.254271,-160.152142,401.718242,12.149771,-160.126092,15.757091,"[10, 401, 401, 253, 6, 7]","[[279, 2], [11199, 7], [11199, 1], [447, 0], [..."


In [ ]:
'''
Z-score database 

TO DO:
- Count all the network motifs in real network 
- Generate 100-1000 random networks 
- Count all the network motifs in random networks 
- Calculate average number of motifs and standard-deviation 
- Calculate z-score 
'''

zscoreDF = pd.DataFrame(columns=['Sparsity Index', 'Masks',
                                '1-O motifs (real)', 
                                'S-O diverging motifs (real)', 'S-O converging motifs (real)', 
                                'S-O chain motifs (real)',  'T-O chain motifs (real)',
                                'T-O diverging motifs (real)', 'T-O converging motifs (real)',
                                
                                'Avg - S-O diverging motifs (random)', 'Avg - S-O converging motifs (random)', 
                                'Avg - S-O chain motifs (random)',  'Avg - T-O chain motifs (random)',
                                'Avg - T-O diverging motifs (random)', 'Avg - T-O converging motifs (random)',
                                 
                                'SD - S-O diverging motifs (random)', 'SD - S-O converging motifs (random)', 
                                'SD - S-O chain motifs (random)',  'SD - T-O chain motifs (random)',
                                'SD - T-O diverging motifs (random)', 'SD - T-O converging motifs (random)',
                                
                                'Z - S-O diverging motifs', 'Z - S-O converging motifs', 
                                'Z - S-O chain motifs',  'Z - T-O chain motifs',
                                'Z - T-O diverging motifs', 'Z - T-O converging motifs',
                                'Number of nodes in each layer with downstream output', 
                                'Number of connections in each layer'])
#TESTING ON FIRST FIVE NETWORKS
for (sparsity, m) in masksTest:
    '''
    Calculate number of first-order and second-order motifs. 
    '''
    numFC = [0,0,0,0,0,0] #Number of remaining nodes with downstream output
    numUS = [0,0,0,0,0] #Number of remaining nodes with upstream input (skip first layer)
    FOM = 0
    FOMList = [[0,0],[0,0],[0,0],[0,0],[0,0]] #[[Num weights, num biases]] in each layer
    SODM = 0
    SOCM = 0 
    TODM = 0 
    TOCM = 0
    SOChain = 0
    TOChain = 0
    for i in range(len(m)): 
        #Calculate first-order motifs

        #Count number of connections between weights
        w_connections = np.count_nonzero(m[i][0:-1])
        FOMList[i][0] = w_connections
        #Count number of connections from bias
        b_connections = np.count_nonzero(m[i][-1])
        FOMList[i][1] = b_connections

        connections = w_connections + b_connections
        FOM += connections
        
        
        nodes = 0
        #Calculate second-order diverging motifs
        for row in m[i]:
            n = np.count_nonzero(row)
            if n >= 2:
                SODM += math.factorial(n)/(math.factorial(2)*math.factorial(n-2))
                
            #also calculate number of remaining nodes
            if n > 0: 
                nodes += 1
                
        numFC[i] = nodes
        
        #Count number of nodes in final layer 
        if i == 4: 
            nodes = 0 
            for row in m[i].T:
                n = np.count_nonzero(row)
                if n > 0 :
                    nodes += 1
            numFC[i+1] = nodes
        
        nodesUS = 0
        #Calculate second-order converging motifs
        for column in m[i].T:
            n = np.count_nonzero(column)
            if n >= 2:
                SOCM += math.factorial(n)/(math.factorial(2)*math.factorial(n-2))
            
            #if i >= 1: 
            #   #also calculate number of remaining nodes
             #   if n > 0: 
             #       nodesUS += 1
             #   numUS[i-1] = nodesUS
                
        #Calculate second-order chain motifs
        if i != 4: 
            #Exclude the bias by excluding the last row
            SOChain += np.count_nonzero(np.matmul(m[i][0:-1],m[i+1][0:-1]))
            
            #Add in the motifs from the bias terms 
            SOChain += np.count_nonzero(np.matmul(m[i][-1],m[i+1][0:-1]))
        else: 
            pass


        #Calculate third-order diverging motifs
        for row in m[i]:
            n = np.count_nonzero(row)
            if n >= 3:
                TODM += math.factorial(n)/(math.factorial(3)*math.factorial(n-3))

        #Calculate third-order converging motifs 
        for column in m[i].T:
            n = np.count_nonzero(column)
            if n >= 3:
                TOCM += math.factorial(n)/(math.factorial(2)*math.factorial(n-2))

        #Calculate third-order chain motifs 
        if i in (0,1,2): 
            #Count non-zero elements of (Layer 1 * Layer 2 * Layer 3)
            #Exclude the bias by excluding the last row
            m1 = np.matmul(m[i][0:-1],m[i+1][0:-1])
            TOChain += np.count_nonzero(np.matmul(m1,m[i+2][0:-1]))
            
            #Add in the motifs from the bias terms 
            mbias = np.matmul(m[i][-1],m[i+1][0:-1])
            TOChain += np.count_nonzero(np.matmul(mbias,m[i+2][0:-1]))
        else: 
            pass

    '''
    Build 1000 randomly connected sparse networks with same number of connections
    between weights and biases.

    TO DO: 
    - Check how many new bias terms are generated in the randomization.
    - Calculate average motif counts 
    - Outside of this loop, calculate z-score. 
     

    ''' 
    randomNetDF = pd.DataFrame(columns=['rSODM',
        'rSOCM',
        'rSOChain',
        'rTODM',
        'rTOCM', 
        'rTOChain'])

    for r in range(1000):
        
        #random weight matrix = np.array(([1]*num connections between weights)+
        #                       [0]*(num possible connections - num connections between weights))
        #           numFC[0]-1 because we need to discount bias 
        r1 = np.array([1] * (FOMList[0][0]) + [0] * (((numFC[0]-1)*(numFC[1]-1))-(FOMList[0][0])))
        #random bias matrix = np.array(([1]*num connections between bias and next nodes)+
        #                       [0]*(num possible connections - num connections between bias and next nodes))

        #There is always a live bias, so number of possible connections between bias and next
        #   layer would be numFC[i+1]-1 (to remove bias).
        r1b = np.array([1] * FOMList[0][1] + [0] * ((numFC[1]-1)-FOMList[0][1]))

        r2 = np.array([1] * (FOMList[1][0]) + [0] * (((numFC[1]-1)*(numFC[2]-1))-(FOMList[1][0])))
        r3 = np.array([1] * (FOMList[2][0]) + [0] * (((numFC[2]-1)*(numFC[3]-1))-(FOMList[2][0])))
        r4 = np.array([1] * (FOMList[3][0]) + [0] * (((numFC[3]-1)*(numFC[4]-1))-(FOMList[3][0])))
        r5 = np.array([1] * (FOMList[4][0]) + [0] * (((numFC[4]-1)*(numFC[5]))-(FOMList[4][0]))) #no bias in last layer

        r2b = np.array([1] * FOMList[1][1] + [0] * ((numFC[2]-1)-FOMList[1][1]))
        r3b = np.array([1] * FOMList[2][1] + [0] * ((numFC[3]-1)-FOMList[2][1]))
        r4b = np.array([1] * FOMList[3][1] + [0] * ((numFC[4]-1)-FOMList[3][1]))
        r5b = np.array([1] * FOMList[4][1] + [0] * ((numFC[5])-FOMList[4][1]))
        
        np.random.shuffle(r1)
        np.random.shuffle(r2)
        np.random.shuffle(r3)
        np.random.shuffle(r4)
        np.random.shuffle(r5)

        np.random.shuffle(r1b)
        np.random.shuffle(r2b)
        np.random.shuffle(r3b)
        np.random.shuffle(r4b)
        np.random.shuffle(r5b)
            
        randomNet = [np.vstack([np.reshape(r1, (((numFC[0]-1),(numFC[1]-1)))),r1b]),
                    np.vstack([np.reshape(r2, (((numFC[1]-1),(numFC[2]-1)))),r2b]),
                    np.vstack([np.reshape(r3, (((numFC[2]-1),(numFC[3]-1)))),r3b]),
                    np.vstack([np.reshape(r4, (((numFC[3]-1),(numFC[4]-1)))),r4b]),
                    np.vstack([np.reshape(r5, (((numFC[4]-1),(numFC[5])))),r5b])]

        '''
        Count how many bias terms are added to the random network 

        Conclusion: Bias terms are added to first layer because we are going from low node to high node layer. 
        TO DO: 
        - Prove that if nodes in in-layer < nodes in out-layer that number of added biases = 0 
        - Make sure that in randomly gerenated networks, there are no added bias terms
            - Actually, there are still added bias terms in the first layer of the real network. Why??


        - If the number of connections between layers/number of nodes in out-layer > number of nodes in out-layer then
        there is some probability that each node in the out-layer will have an upstream input 
        '''
        addedBias = [0, 0, 0, 0, 0] #Number of bias terms in random network

        #Iterate over the layers in the network 
        for i in range(len(randomNet)-1):
            #Iterate over the columns 
            for j in range(len(randomNet[i].T)):
                column = randomNet[i].T[j]
                #Check to see if there are any connections between this node and the nodes 
                # in the previous layer. 
                up = np.count_nonzero(column)

                #If up == 0, that means there are no upstream connections 
                # and this might be an added bias.
                if up == 0:
                    #Check to see if there are any downstream connections.
                    row = randomNet[i+1][j]
                    down = np.count_nonzero(row)
                    #print((i,up,down))
                    #If down != 0, that means there are downstream connections 
                    # and this is an added bias. 
                    if down != 0:
                        #Add to count of added biases for this layer
                        addedBias[i] += 1
    
        rSODM = 0
        rSOCM = 0 
        rSOChain = 0
        rTODM = 0
        rTOCM = 0 
        rTOChain = 0

        for j in range(len(randomNet)):
            
            #Calculate second-order diverging motifs in random network
            for row in randomNet[j]:
                n = np.count_nonzero(row)
                if n >= 2:
                    rSODM += math.factorial(n)/(math.factorial(2)*math.factorial(n-2))
            
            #Calculate second-order converging motifs in random network
            for column in randomNet[j].T:
                n = np.count_nonzero(column)
                if n >= 2:
                    rSOCM += math.factorial(n)/(math.factorial(2)*math.factorial(n-2))
                    
            #Calculate second-order chain motifs in random network
            if j != 4: 
                #Exclude the bias by excluding the last row
                rSOChain += np.count_nonzero(np.matmul(randomNet[j][0:-1],randomNet[j+1][0:-1]))
                
                #Add in the motifs from the bias terms 
                rSOChain += np.count_nonzero(np.matmul(randomNet[j][-1],randomNet[j+1][0:-1]))
            else: 
                pass 

            #Calculate third-order diverging motifs in random network
            for row in randomNet[j]:
                n = np.count_nonzero(row)
                if n >= 3:
                    rTODM += math.factorial(n)/(math.factorial(3)*math.factorial(n-3))
            
            #Calculate third-order converging motifs in random network
            for column in randomNet[j].T:
                n = np.count_nonzero(column)
                if n >= 3:
                    rTOCM += math.factorial(n)/(math.factorial(3)*math.factorial(n-3))
                    
            #Calculate third-order chain motifs in random network
            if j in (0,1,2):
                #Count non-zero elements of (Layer 1 * Layer 2 * Layer 3)
                #Exclude the bias by excluding the last row
                m1r = np.matmul(randomNet[j][0:-1],randomNet[j+1][0:-1])
                rTOChain += np.count_nonzero(np.matmul(m1r,randomNet[j+2][0:-1]))
                
                #Add in the motifs from the bias terms 
                mbiasr = np.matmul(randomNet[j][-1],randomNet[j+1][0:-1])
                rTOChain += np.count_nonzero(np.matmul(mbiasr,randomNet[j+2][0:-1]))
            else: 
                pass  

        rMotifs = [float(rSODM),
            float(rSOCM),
            float(rSOChain),
            float(rTODM),
            float(rTOCM),
            float(rTOChain)]

        randomNetDF.loc[len(randomNetDF.index)] = rMotifs

    AvgrSODM = randomNetDF['rSODM'].mean()
    AvgrSOCM = randomNetDF['rSOCM'].mean()
    AvgrSOChain = randomNetDF['rSOChain'].mean()
    AvgrTODM = randomNetDF['rTODM'].mean()
    AvgrTOCM = randomNetDF['rTOCM'].mean()
    AvgrTOChain = randomNetDF['rTOChain'].mean()

    SDrSODM = randomNetDF['rSODM'].std()
    SDrSOCM = randomNetDF['rSOCM'].std()
    SDrSOChain = randomNetDF['rSOChain'].std()
    SDrTODM = randomNetDF['rTODM'].std()
    SDrTOCM = randomNetDF['rTOCM'].std()
    SDrTOChain = randomNetDF['rTOChain'].std()

    ZSODM = (SODM - AvgrSODM)/SDrSODM
    ZSOCM = (SOCM - AvgrSOCM)/SDrSOCM
    ZSOChain = (SOChain - AvgrSOChain)/SDrSOChain
    ZTODM = (TODM - AvgrTODM)/SDrTODM
    ZTOCM = (TOCM - AvgrTOCM)/SDrTOCM
    ZTOChain = (TOChain - AvgrTOChain)/SDrTOChain

    zscoreData = [float(sparsity), m, 
                    float(FOM), 
                    float(SODM), float(SOCM), float(SOChain),
                    float(TODM), float(TOCM), float(TOChain),

                    float(AvgrSODM), float(AvgrSOCM), float(AvgrSOChain),
                    float(AvgrTODM), float(AvgrTOCM), float(AvgrTOChain),

                    float(SDrSODM), float(SDrSOCM), float(SDrSOChain),
                    float(SDrTODM), float(SDrTOCM), float(SDrTOChain),

                    float(ZSODM), float(ZSOCM), float(ZSOChain),
                    float(ZTODM), float(ZTOCM), float(ZTOChain),

                    numFC, FOMList]

    zscoreDF.loc[len(zscoreDF.index)] = zscoreData
    

    
        